In [1]:
import pandas as pd
from tqdm.auto import tqdm
from ast import literal_eval
from pymatgen.ext.matproj import MPRester
from pymatgen.core import Composition

from htvs_module.htvs_mods import substitution, comp_to_form

mpr = MPRester('WgPaLVntnoS3X3tdlT')

# Li containing materials from Materials Project

In [2]:
data = mpr.query({"elements":'Li','nelements':{ "$gte" : 3 , "$lte" : 3}}, properties=['material_id',"pretty_formula",'elements','formula','cif'])
df = pd.DataFrame(data)
df.to_csv('./data/01_1_all_Li_material_from_MP.csv',index=False)

  0%|          | 0/5037 [00:00<?, ?it/s]

In [3]:
df = pd.read_csv('./data/01_1_all_Li_material_from_MP.csv')[:10]
df

,material_id,pretty_formula,elements,formula,cif
0,mp-1001784,LiTiS2,"['Li', 'Ti', 'S']","{'Li': 1.0, 'Ti': 1.0, 'S': 2.0}",# generated using pymatgen\ndata_LiTiS2\n_symm...
1,mp-1014462,Li2YAl,"['Li', 'Y', 'Al']","{'Li': 2.0, 'Y': 1.0, 'Al': 1.0}",# generated using pymatgen\ndata_Li2YAl\n_symm...
2,mp-1015061,NaLi2Sn,"['Li', 'Na', 'Sn']","{'Na': 1.0, 'Li': 2.0, 'Sn': 1.0}",# generated using pymatgen\ndata_NaLi2Sn\n_sym...
3,mp-1018764,Li3HoSb2,"['Li', 'Ho', 'Sb']","{'Li': 3.0, 'Ho': 1.0, 'Sb': 2.0}",# generated using pymatgen\ndata_Li3HoSb2\n_sy...
4,mp-1025496,LiNbSe2,"['Li', 'Nb', 'Se']","{'Li': 1.0, 'Nb': 1.0, 'Se': 2.0}",# generated using pymatgen\ndata_LiNbSe2\n_sym...
5,mp-1029331,LiHf2N3,"['Hf', 'Li', 'N']","{'Li': 1.0, 'Hf': 2.0, 'N': 3.0}",# generated using pymatgen\ndata_LiHf2N3\n_sym...
6,mp-1029395,LiCrN2,"['Li', 'Cr', 'N']","{'Li': 1.0, 'Cr': 1.0, 'N': 2.0}",# generated using pymatgen\ndata_LiCrN2\n_symm...
7,mp-10616,BaLiAs,"['Ba', 'Li', 'As']","{'Ba': 1.0, 'Li': 1.0, 'As': 1.0}",# generated using pymatgen\ndata_BaLiAs\n_symm...
8,mp-10686,Li2CdGe,"['Li', 'Cd', 'Ge']","{'Li': 2.0, 'Cd': 1.0, 'Ge': 1.0}",# generated using pymatgen\ndata_Li2CdGe\n_sym...
9,mp-10687,LiCdP,"['Li', 'Cd', 'P']","{'Li': 1.0, 'Cd': 1.0, 'P': 1.0}",# generated using pymatgen\ndata_LiCdP\n_symme...


In [4]:
for i in tqdm(range(len(df))): 
    formula_dict = literal_eval(df.iloc[i]['formula'])
    df.iloc[i]['formula'] = (list(formula_dict.keys())+list(formula_dict.values()))

  0%|          | 0/10 [00:00<?, ?it/s]

## Selected element for structure generation

In [5]:
sub_el_list = ['Li','B','C','N','O','F','Na','Mg','Al','Si','P','S',
               'Cl','Ca','Zn','Ga','Ge','As','Se','Sr','Cd','In',
               'Sn','Te','I','Ba','Hg','Pb'] 

## Elemental substitution

In [6]:
substituted_list=substitution(df,sub_el_list)

  0%|          | 0/10 [00:00<?, ?it/s]

In [7]:
subcomp_mpid_df = pd.DataFrame()
sub_comp_list = []
mpid_list = []
reduced_form_list = []
element = []
for i in tqdm(substituted_list):
    my_comp_list=i[:-1]
    mpid=i[-1]
    reduced_form=Composition(comp_to_form(my_comp_list)).reduced_formula
    
    sub_comp_list.append(my_comp_list)
    mpid_list.append(mpid)
    reduced_form_list.append(reduced_form)
subcomp_mpid_df['sub_comp'] = sub_comp_list
subcomp_mpid_df['mpid'] = mpid_list
subcomp_mpid_df['reduced_formula'] = reduced_form_list
subcomp_mpid_df

  0%|          | 0/560 [00:00<?, ?it/s]

,sub_comp,mpid,reduced_formula
0,"[Li, S, Li, 1.0, 1.0, 2.0]",mp-1001784,Li3S
1,"[Li, S, B, 1.0, 1.0, 2.0]",mp-1001784,LiB2S
2,"[Li, S, C, 1.0, 1.0, 2.0]",mp-1001784,LiC2S
3,"[Li, S, N, 1.0, 1.0, 2.0]",mp-1001784,LiSN2
4,"[Li, S, O, 1.0, 1.0, 2.0]",mp-1001784,LiSO2
...,...,...,...
555,"[Li, Te, S, 1.0, 1.0, 1.0]",mp-10687,LiTeS
556,"[Li, I, S, 1.0, 1.0, 1.0]",mp-10687,LiSI
557,"[Li, Ba, S, 1.0, 1.0, 1.0]",mp-10687,BaLiS
558,"[Li, Hg, S, 1.0, 1.0, 1.0]",mp-10687,LiHgS


In [8]:
subcomp_mpid_df.to_csv('./data/01_2_substitution_final.csv',index=False)
subcomp_mpid_df = pd.read_csv('./data/01_2_substitution_final.csv')
subcomp_mpid_df

,sub_comp,mpid,reduced_formula
0,"['Li', 'S', 'Li', 1.0, 1.0, 2.0]",mp-1001784,Li3S
1,"['Li', 'S', 'B', 1.0, 1.0, 2.0]",mp-1001784,LiB2S
2,"['Li', 'S', 'C', 1.0, 1.0, 2.0]",mp-1001784,LiC2S
3,"['Li', 'S', 'N', 1.0, 1.0, 2.0]",mp-1001784,LiSN2
4,"['Li', 'S', 'O', 1.0, 1.0, 2.0]",mp-1001784,LiSO2
...,...,...,...
555,"['Li', 'Te', 'S', 1.0, 1.0, 1.0]",mp-10687,LiTeS
556,"['Li', 'I', 'S', 1.0, 1.0, 1.0]",mp-10687,LiSI
557,"['Li', 'Ba', 'S', 1.0, 1.0, 1.0]",mp-10687,BaLiS
558,"['Li', 'Hg', 'S', 1.0, 1.0, 1.0]",mp-10687,LiHgS
